In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_max_distance_to_target_surface, contact_optimization, show_selected_joints)

import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
rod_length = 0.05979705114046488
width = rod_length / 20 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('bird_close_beak_1', thickness, width, weaving_dir)

In [ ]:
import py_newton_optimizer
# Optimization parameters.
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 10
drw = 0.01
dsw = 0.1

In [ ]:
import pickle
import gzip

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width = True)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
curved_linkage.setExtendedDoFsPSRL(np.load('../../nature_paper_experiments/figure_1/results/bird_close_beak_1/bird_close_beak_1_dof.npy'))

In [ ]:
curved_linkage.attraction_weight = 1e-5

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update()

In [ ]:
OPTS.niter = 10000

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

In [ ]:
from structural_analysis import weavingCrossingForceMagnitudes
cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
np.argsort(cfm[:, 0])

### Deactivate Crossing

In [ ]:
import deactivate_crossing_helper
importlib.reload(deactivate_crossing_helper)
from deactivate_crossing_helper import (write_deactivated_linkage, copy_over_under, get_segment_index_from_joints, get_angle_from_kappa, get_kappa_from_angle, no_resample_deactivation, resample_deactivation, get_neighbors)

In [ ]:
def create_deactivated_linkage(original_linkage, deactive_index, original_model_name, deactivated_model_name):
    ju = original_linkage.joint(deactive_index)
    nbs = ju.neighbors()
    print(nbs)
    deactive_vertex_map = [i if i < deactive_index else i - 1 for i in range(original_linkage.numJoints())]
    write_deactivated_linkage(original_model_name, deactivated_model_name, deactive_index, deactive_vertex_map, nbs)
    
    with so(): deactivate_curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = deactivated_model_name, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=elastic_rods.InterleavingType.weaving)
    deactivate_curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    deactivate_curved_linkage.attraction_weight = 1e-5

#     Copy the over under assignment from the original model since BFS will not find a perfect over under in the new topology
    copy_over_under(deactivate_curved_linkage, original_linkage, deactive_vertex_map, deactive_index)
    deactivate_curved_linkage.setDoFs(deactivate_curved_linkage.getDoFs())

    resample_deactivation(deactivate_curved_linkage, original_linkage, deactive_vertex_map, nbs, deactive_index, io)
    return deactivate_curved_linkage, deactive_vertex_map, nbs

In [ ]:
deactive_index = 90

In [ ]:
deactivated_model_name = ('/'.join(io.MODEL_PATH.split('/')[:-1] + [io.MODEL_NAME + '_deactive_joint_{}.obj'.format(deactive_index)]))

In [ ]:
deactivate_linkage, deactive_vertex_map, nbs = create_deactivated_linkage(curved_linkage, deactive_index, io.MODEL_PATH, deactivated_model_name)

In [ ]:
deactivate_curved_linkage_view = linkage_vis.LinkageViewer(deactivate_linkage)
deactivate_curved_linkage_view.show()

In [ ]:
topColor =[150/255., 150/255., 150/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = deactivate_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [ ]:
def eqm_callback(prob, i):
    deactivate_curved_linkage_view.update()

In [ ]:
elastic_rods.compute_equilibrium(deactivate_linkage, callback = eqm_callback, options = OPTS)

In [ ]:
vf = show_selected_joints(deactivate_linkage, [deactive_vertex_map[nbs[0]], deactive_vertex_map[nbs[1]], deactive_vertex_map[nbs[2]], deactive_vertex_map[nbs[3]]])

In [ ]:
deactivate_curved_linkage_view.update(scalarField = colors, vectorField = vf)